<a href="https://colab.research.google.com/github/FilippMaksimov/Net.FeaturesAndModels/blob/main/Models_Maksimov_F_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
class CustomLogisticRegression():
    def __init__(self, learning_rate=0.01, max_iter=1000):
      self.learning_rate = learning_rate
      self.max_iter = max_iter
      self.W = None
      self.b = None

    def sigmoid(self, x):
      return 1 / (1 + np.exp(-x))

    def fit(self, X, y, method):
      l, n = X.shape
      self.W = np.zeros(n)
      self.b = 0

      for i in range(self.max_iter):
        linear_model = np.dot(X, self.W) + self.b
        y_predicted = self.sigmoid(linear_model)
        #Градиенты
        dw = (1 / l) * np.dot(X.T, (y_predicted - y))
        db = (1 / l) * np.sum(y_predicted - y)

        if method == 'gradient':
          #Обновление параметров
          self.W -= self.learning_rate * dw
          self.b -= self.learning_rate * db

        elif method == 'rmsprop':
          #Параметры для метода скользящего среднего
          gamma = 0.9
          epsilon = 1e-8
          vdw = np.zeros(n)
          vdb = 0

          vdw = gamma * vdw + (1 - gamma) * dw ** 2
          vdb = gamma * vdb + (1 - gamma) * db ** 2
          self.W -= self.learning_rate * dw / (np.sqrt(vdw) + epsilon)
          self.b -= self.learning_rate * db / (np.sqrt(vdb) + epsilon)


        elif method == 'nadam':
          beta1 = 0.9
          beta2 = 0.999
          epsilon = 1e-8
          m = np.zeros(n)
          v = np.zeros(n)
          m_b = 0
          v_b = 0
          t = i + 1

          m = beta1 * m + (1 - beta1) * dw
          v = beta2 * v + (1 - beta2) * (dw ** 2)
          m_b = beta1 * m_b + (1 - beta1) * db
          v_b = beta2 * v_b + (1 - beta2) * (db ** 2)

          m_hat = m / (1 - beta1 ** t)
          v_hat = v / (1 - beta2 ** t)
          m_b_hat = m_b / (1 - beta1 ** t)
          v_b_hat = v_b / (1 - beta2 ** t)

          self.W -= self.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
          self.b -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + epsilon)


    def predict(self, X):
        linear_model = np.dot(X, self.W) + self.b
        y_predicted = self.sigmoid(linear_model)
        return np.where(y_predicted >= 0.5, 1, 0)

    def accuracy(self, y_true, y_pred):
        return np.mean(y_true == y_pred)

In [ ]:
# Загрузка данных
iris = datasets.load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Фильтрация только для классов Iris Versicolor (1) и Iris Virginica (2)
df = df[df['target'].isin([1, 2])]
X = df.iloc[:, :-1]
X['Add'] = 1 #добавление фиктивного признака из единиц
X = X.values
#Замена классов 1 и 2 на 0 и 1 соответственно
target_map = {1: 0, 2: 1}
df['target'] = df['target'].map(target_map)
y = df['target'].values

In [ ]:
import time

# Функция для оценки модели и возврата метрики
def evaluate_model(model_class, X_train, y_train, X_test, y_test, method):
    model = model_class(learning_rate=0.01, max_iter=2000)

    start_time = time.time()
    model.fit(X_train, y_train, method)
    end_time = time.time()

    y_pred = model.predict(X_test)
    accuracy = model.accuracy(y_test, y_pred)
    execution_time = end_time - start_time

    return accuracy, execution_time

In [ ]:
# Сравнение методов
res = []

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Градиентный спуск
accuracy_gd, time_gd = evaluate_model(CustomLogisticRegression, x_train, y_train, x_test, y_test, 'gradient')
res.append(("Градиентный спуск", accuracy_gd, time_gd))

# RMSProp
accuracy_rms, time_rms = evaluate_model(CustomLogisticRegression, x_train, y_train, x_test, y_test, 'rmsprop')
res.append(("RMSProp", accuracy_rms, time_rms))

# Nadam
accuracy_nadam, time_nadam = evaluate_model(CustomLogisticRegression, x_train, y_train, x_test, y_test, 'nadam')
res.append(("Nadam", accuracy_nadam, time_nadam))

# Вывод результатов
print("| Метод          | Метрика (Accuracy) | Время работы (секунд) |")
print("|----------------|--------------------|-----------------------|")
for method, accuracy, exec_time in res:
    print(f"| {method:<15} | {accuracy:.4f}           | {exec_time:.4f}               |")

| Метод          | Метрика (Accuracy) | Время работы (секунд) |
|----------------|--------------------|-----------------------|
| Градиентный спуск | 0.9000           | 0.0479               |
| RMSProp         | 0.9000           | 0.0666               |
| Nadam           | 0.8500           | 0.0787               |
